In [ ]:
import re
import pandas as pd
from tqdm import tqdm
from pykospacing import Spacing

import rhinoMorph
from sklearn.feature_extraction.text import CountVectorizer

# tqdm과 pandas 통합
tqdm.pandas()

# Rhino 객체 초기화
rn = rhinoMorph.startRhino()

In [ ]:
spacing = Spacing()

def preprocessing(text):
    text = spacing(text)
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# 불용어 리스트 가져오기
def get_stopwords():
    stopwords = set()
    with open('../../../data/stopwords-ko.txt', 'r', encoding='utf-8') as file:
        for line in file:
            stopwords.add(line.strip())
    return stopwords

# 형태소 분석 및 불용어 제거 함수
def morph_and_remove_stopwords(text, stopwords):
    text = preprocessing(text)
    morphs = rhinoMorph.onlyMorph_list(rn, text)
    meaningful_words = [word for word in morphs if word not in stopwords]
    return meaningful_words

In [ ]:
# 불용어 리스트 가져오기
stopwords = get_stopwords()

df = pd.read_csv(r'..\..\..\data\combined_questions.csv', encoding='cp949')

# 데이터프레임의 'feature' 열에 대해 형태소 분석 및 불용어 제거 적용
df['processed_feature'] = df['feature'].apply(lambda x: morph_and_remove_stopwords(x, stopwords))

df.head()

In [ ]:
df['processed_feature_str'] = df['processed_feature'].apply(lambda x: ' '.join(x))  # processed_feature를 문자열로 변환

vectorizer = CountVectorizer()                                                      # CountVectorizer를 사용하여 BoW 벡터화
X = vectorizer.fit_transform(df['processed_feature_str'])
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())      # BoW 결과를 데이터프레임으로 변환

df_bow = pd.concat([df, bow_df], axis=1)                                            # 원래 데이터프레임과 BoW 데이터프레임을 병합